In [ ]:
# Import libraries
import numpy as np
from sklearn.linear_model import LinearRegression, RANSACRegressor, Ridge, Lasso
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold
from sklearn.mixture import GaussianMixture
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Load Data Arrays
x = np.load('X_train_regression2.npy')
y = np.load('y_train_regression2.npy')
xx = np.load('X_test_regression2.npy')

In [ ]:
# Linear Regression with cross validation (loo)
loo = LeaveOneOut()
predicted_values = []
sse_values = []
coef_values = []
for train_index, test_index in loo.split(x):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]

    lregr = LinearRegression()
    lregr.fit(x_train, y_train)

    y_pred = lregr.predict(x_test)
    sse_values.append(np.mean((y_test - y_pred)**2))

    coef_values.append(lregr.coef_)
    predicted_values.append(y_pred.flatten())

mean_coef = np.mean(coef_values, axis=0)
mse = np.mean((y-predicted_values)**2)
var_y = sum((y - np.mean(y))**2)/len(y)
r_squared = 1 - mse/var_y

print(mean_coef[0], f"\nmse is: {mse}", f"\nr^2 is: {r_squared.flatten()}")

In [ ]:
for i in range(0,4):
    counts, bins = np.histogram(x[:,i])
    plt.hist(bins[:-1], bins, weights=counts)

In [ ]:
# Using Gaussian Mixer, with linear regression
data = np.concatenate([x,y],axis=1)
gm = GaussianMixture(n_components=2, covariance_type='full', init_params='kmeans', random_state=69).fit(data)
mask = gm.predict(data)
cluster_1 = np.where(mask == 1)[0]
cluster_2 = np.where(mask == 0)[0]
print(f"The cluster sizes are {len(cluster_1)} / {len(cluster_2)} respectively.")

x_cluster1, y_cluster1 = x[cluster_1], y[cluster_1]
x_cluster2, y_cluster2 = x[cluster_2], y[cluster_2]

print("===== Model 1 =====")
kf = KFold(n_splits=15, shuffle=True, random_state=0)
predicted_values = []
sse_values = []
coef_values = []
for train_index, test_index in kf.split(x_cluster1):
    x_train, x_test = x_cluster1[train_index], x_cluster1[test_index]
    y_train, y_test = y_cluster1[train_index], y_cluster1[test_index]

    lregr = LinearRegression()
    lregr.fit(x_train, y_train)

    y_pred = lregr.predict(x_test)
    sse_values.append(np.mean((y_test - y_pred)**2))

    coef_values.append(lregr.coef_)
    predicted_values.append(y_pred.flatten())
    
sse = np.mean(sse_values)
ss_tot = sum((y_cluster1 - np.mean(y_cluster1))**2)/len(y_cluster1)
r_squared = 1 - sse/ss_tot

mean_coef_1 = np.mean(coef_values, axis=0)
print("Mean coefs", mean_coef_1)
print("mse:",sse, "\nss_tot:",ss_tot, "\nr_squared:",r_squared)


print("===== Model 2 =====")
kf2 = KFold(n_splits=15, shuffle=True, random_state=0)
predicted_values = []
sse_values = []
coef_values = []

for train_index, test_index in kf2.split(x_cluster2):
    x_train, x_test = x_cluster2[train_index], x_cluster2[test_index]
    y_train, y_test = y_cluster2[train_index], y_cluster2[test_index]

    lregr = LinearRegression()
    lregr.fit(x_train, y_train)

    y_pred = lregr.predict(x_test)
    sse_values.append(np.mean((y_test - y_pred)**2))

    coef_values.append(lregr.coef_)
    predicted_values.append(y_pred.flatten())

sse = np.mean(sse_values)
ss_tot = sum((y_cluster2 - np.mean(y_cluster2))**2)/len(y_cluster2)
r_squared = 1 - sse/ss_tot

mean_coef_2 = np.mean(coef_values, axis=0)
print("Mean coefs", mean_coef_2)
print("mse:",sse, "\nss_tot:",ss_tot, "\nr_squared:",r_squared)

# Generating output array


lregr_1 = LinearRegression()
lregr_1.fit(x_cluster1, y_cluster1)
y_pred_1 = lregr_1.predict(xx)
lregr_2 = LinearRegression()
lregr_2.fit(x_cluster2, y_cluster2)
y_pred_2 = lregr_2.predict(xx)
y_out = np.concatenate([y_pred_1,y_pred_2], axis=1)
np.save('Y_test_regression2.npy', y_out)

In [ ]:
for i in range(0,4):
    counts, bins = np.histogram(x_cluster1[:,i])
    plt.hist(bins[:-1], bins, weights=counts)

In [ ]:
for i in range(0,4):
    counts, bins = np.histogram(x_cluster2[:,i])
    plt.hist(bins[:-1], bins, weights=counts)

In [ ]:
# Using Gaussian Mixer with Ridge
data = np.concatenate([x,y],axis=1)
gm = GaussianMixture(n_components=2, covariance_type='full', init_params='kmeans', random_state=42).fit(data)
mask = gm.predict(data)
cluster_1 = np.where(mask == 1)[0]
cluster_2 = np.where(mask == 0)[0]
print(f"The cluster sizes are {len(cluster_1)} / {len(cluster_2)} respectively.")

x_cluster1, y_cluster1 = x[cluster_1], y[cluster_1]
x_cluster2, y_cluster2 = x[cluster_2], y[cluster_2]

print("===== Model 1 =====")
kf = KFold(n_splits=15, shuffle=True, random_state=47)
coef_values = []
mse_values = []
mean_coef = []
alpha_min = 0
alphas = np.logspace(-4,-2,5000)
for alpha in alphas:
    sse_values = []
    coef_values = []
    for train_index, test_index in kf.split(x_cluster1):
        x_train, x_test = x_cluster1[train_index], x_cluster1[test_index]
        y_train, y_test = y_cluster1[train_index], y_cluster1[test_index]

        ridge = Ridge(alpha=alpha, random_state=0)
        ridge.fit(x_train, y_train)

        y_pred = ridge.predict(x_test)
        sse_fold = np.sum((y_test - y_pred)**2)
        sse_values.append(sse_fold)
        coef_values.append(ridge.coef_)
    mse_values.append(np.mean(sse_values))
    mean_coef.append(np.mean(coef_values, axis=0))
    
alpha_min = alphas[np.array(mse_values).argmin()]
ss_tot = sum((y_cluster1 - np.mean(y_cluster1))**2)/len(y_cluster1)
r_squared = 1 - np.min(mse_values)/ss_tot
print("Mean coefs", mean_coef[np.where(alphas == alpha_min)[0][0]])
print("best alpha:",alpha_min,"mse:",np.min(mse_values), "\nss_tot:",ss_tot, "\nr_squared:",r_squared.flatten())

print("===== Model 2 =====")
kf2 = KFold(n_splits=15, shuffle=True, random_state=49)
coef_values = []
mse_values = []
mean_coef = []
alpha_min = 0
alphas = np.logspace(-1,0,5000)
for alpha in alphas:
    sse_values = []
    coef_values = []
    for train_index, test_index in kf2.split(x_cluster2):
        x_train, x_test = x_cluster2[train_index], x_cluster2[test_index]
        y_train, y_test = y_cluster2[train_index], y_cluster2[test_index]

        ridge = Ridge(alpha=alpha, random_state=0)
        ridge.fit(x_train, y_train)

        y_pred = ridge.predict(x_test)
        sse_fold = np.sum((y_test - y_pred)**2)
        sse_values.append(sse_fold)
        coef_values.append(ridge.coef_)
    mse_values.append(np.mean(sse_values))
    mean_coef.append(np.mean(coef_values, axis=0))
    
alpha_min = alphas[np.array(mse_values).argmin()]
ss_tot = sum((y_cluster2 - np.mean(y_cluster2))**2)/len(y_cluster2)
r_squared = 1 - np.min(mse_values)/ss_tot
print("Mean coefs", mean_coef[np.where(alphas == alpha_min)[0][0]])
print("best alpha:",alpha_min,"mse:",np.min(mse_values), "\nss_tot:",ss_tot, "\nr_squared:",r_squared.flatten())

In [ ]:
# Using Gaussian Mixer with Lasso
data = np.concatenate([x,y],axis=1)
gm = GaussianMixture(n_components=2, covariance_type='full', init_params='kmeans', random_state=42).fit(data)
mask = gm.predict(data)
cluster_1 = np.where(mask == 1)[0]
cluster_2 = np.where(mask == 0)[0]
print(f"The cluster sizes are {len(cluster_1)} / {len(cluster_2)} respectively.")

x_cluster1, y_cluster1 = x[cluster_1], y[cluster_1]
x_cluster2, y_cluster2 = x[cluster_2], y[cluster_2]

print("===== Model 1 =====")
kf = KFold(n_splits=2, shuffle=True, random_state=47)
coef_values = []
mse_values = []
mean_coef = []
alpha_min = 0
alphas = np.logspace(-2,2,10_000)
for alpha in alphas:
    sse_values = []
    coef_values = []
    for train_index, test_index in kf.split(x_cluster1):
        x_train, x_test = x_cluster1[train_index], x_cluster1[test_index]
        y_train, y_test = y_cluster1[train_index], y_cluster1[test_index]

        ridge = Lasso(alpha=alpha, random_state=0)
        ridge.fit(x_train, y_train)

        y_pred = ridge.predict(x_test)
        sse_fold = np.sum((y_test - y_pred)**2)
        sse_values.append(sse_fold)
        coef_values.append(ridge.coef_)
    mse_values.append(np.mean(sse_values))
    mean_coef.append(np.mean(coef_values, axis=0))
    
alpha_min = alphas[np.array(mse_values).argmin()]
ss_tot = sum((y_cluster1 - np.mean(y_cluster1))**2)/len(y_cluster1)
r_squared = 1 - np.min(mse_values)/ss_tot
print("Mean coefs", mean_coef[np.where(alphas == alpha_min)[0][0]])
print("best alpha:",alpha_min,"sse:",np.min(mse_values), "\nss_tot:",ss_tot, "\nr_squared:",r_squared.flatten())

print("===== Model 2 =====")
kf2 = KFold(n_splits=2, shuffle=True, random_state=49)
coef_values = []
mse_values = []
mean_coef = []
alpha_min = 0
alphas = np.logspace(-2,2,10_000)
for alpha in alphas:
    sse_values = []
    coef_values = []
    for train_index, test_index in kf2.split(x_cluster2):
        x_train, x_test = x_cluster2[train_index], x_cluster2[test_index]
        y_train, y_test = y_cluster2[train_index], y_cluster2[test_index]

        ridge = Lasso(alpha=alpha, random_state=0)
        ridge.fit(x_train, y_train)

        y_pred = ridge.predict(x_test)
        sse_fold = np.sum((y_test - y_pred)**2)
        sse_values.append(sse_fold)
        coef_values.append(ridge.coef_)
    mse_values.append(np.mean(sse_values))
    mean_coef.append(np.mean(coef_values, axis=0))
    
alpha_min = alphas[np.array(mse_values).argmin()]
ss_tot = sum((y_cluster2 - np.mean(y_cluster2))**2)/len(y_cluster2)
r_squared = 1 - np.min(mse_values)/ss_tot
print("Mean coefs", mean_coef[np.where(alphas == alpha_min)[0][0]])
print("best alpha:",alpha_min,"sse:",np.min(mse_values), "\nss_tot:",ss_tot, "\nr_squared:",r_squared.flatten())

In [ ]:
#Ransac Regressor, with linear regression as estimator and kfold cross validation

ransac = RANSACRegressor(estimator = LinearRegression(), residual_threshold=0.7 ,min_samples=15, max_trials=500_000, random_state=42)
ransac.fit(x, y)

inlier_mask = ransac.inlier_mask_
inliers = [i for i, x in enumerate(inlier_mask) if x == True]
outliers = [i for i, x in enumerate(inlier_mask) if x == False]
print("Inliers:",inliers)
print("Outliers:",outliers)

x_cluster1, y_cluster1 = x[inliers], y[inliers]
x_cluster2, y_cluster2 = x[outliers], y[outliers]

print("Cluster 1:",x_cluster1.shape, y_cluster1.shape)
print("Cluster 2:",x_cluster2.shape, y_cluster2.shape)

# Linear Regression with cross validation (kfold)
print("===== Model 1 =====")
kf = KFold(n_splits=15, shuffle=True, random_state=47)
predicted_values = []
sse_values = []
coef_values = []
for train_index, test_index in kf.split(x_cluster1):
    x_train, x_test = x_cluster1[train_index], x_cluster1[test_index]
    y_train, y_test = y_cluster1[train_index], y_cluster1[test_index]

    lregr = LinearRegression()
    lregr.fit(x_train, y_train)
    y_pred = lregr.predict(x_test)
    sse_values.append(np.mean((y_test - y_pred)**2))

    coef_values.append(lregr.coef_)
    predicted_values.append(y_pred.flatten())
    table = np.column_stack((y_test, y_pred))
    
sse = np.mean(sse_values)
ss_tot = sum((y_cluster1 - np.mean(y_cluster1))**2)/len(y_cluster1)
r_squared = 1 - sse/ss_tot

mean_coef = np.mean(coef_values, axis=0)
print("Mean coefs", mean_coef)
print("sse:",sse, "\nss_tot:",ss_tot, "\nr_squared:",r_squared)


print("===== Model 2 =====")
kf2 = KFold(n_splits=15, shuffle=True, random_state=49)
predicted_values = []
sse_values = []
coef_values = []

for train_index, test_index in kf2.split(x_cluster2):
    x_train, x_test = x_cluster2[train_index], x_cluster2[test_index]
    y_train, y_test = y_cluster2[train_index], y_cluster2[test_index]

    lregr = LinearRegression()
    lregr.fit(x_train, y_train)

    y_pred = lregr.predict(x_test)
    sse_values.append(np.mean((y_test - y_pred)**2))

    coef_values.append(lregr.coef_)
    predicted_values.append(y_pred.flatten())
    table = np.column_stack((y_test, y_pred))

sse = np.mean(sse_values)
ss_tot = sum((y_cluster2 - np.mean(y_cluster2))**2)/len(y_cluster2)
r_squared = 1 - sse/ss_tot

mean_coef = np.mean(coef_values, axis=0)
print("Mean coefs", mean_coef)
print("sse:",sse, "\nss_tot:",ss_tot, "\nr_squared:",r_squared)



In [ ]:
for i in range(0,4):
    counts, bins = np.histogram(x_cluster1[:,i])
    plt.hist(bins[:-1], bins, weights=counts)

In [ ]:
for i in range(0,4):
    counts, bins = np.histogram(x_cluster2[:,i])
    plt.hist(bins[:-1], bins, weights=counts)

In [ ]:
#Ransac Regressor, with linear regression as estimator and kfold cross validation
thresholds = np.linspace(0.5, 1, 100)
r_squared_values_m1 = []
r_squared_values_m2 = []
for threshold in thresholds:
    ransac = RANSACRegressor(estimator = LinearRegression(), residual_threshold=threshold,min_samples=15, max_trials=500_000, random_state=47)
    ransac.fit(x, y)

    inlier_mask = ransac.inlier_mask_
    inliers = [i for i, x in enumerate(inlier_mask) if x == True]
    outliers = [i for i, x in enumerate(inlier_mask) if x == False]

    x_cluster1, y_cluster1 = x[inliers], y[inliers]
    x_cluster2, y_cluster2 = x[outliers], y[outliers]

    # Linear Regression with cross validation (kfold)
    #print("===== Model 1 =====")
    kf = KFold(n_splits=len(x_cluster1), shuffle=True, random_state=47)
    predicted_values = []
    sse_values = []
    coef_values = []

    for train_index, test_index in kf.split(x_cluster1):
        x_train, x_test = x_cluster1[train_index], x_cluster1[test_index]
        y_train, y_test = y_cluster1[train_index], y_cluster1[test_index]
        lregr = LinearRegression()
        lregr.fit(x_train, y_train)

        y_pred = lregr.predict(x_test)
        sse_values.append(np.mean((y_test - y_pred)**2))

        coef_values.append(lregr.coef_)
        predicted_values.append(y_pred.flatten())
        table = np.column_stack((y_test, y_pred))
        
    sse = np.mean(sse_values)
    ss_tot = sum((y_cluster1 - np.mean(y_cluster1))**2)/len(y_cluster1)
    r_squared = 1 - sse/ss_tot
    r_squared_values_m1.append(r_squared)

    mean_coef = np.mean(coef_values, axis=0)


    #print("===== Model 2 =====")
    kf2 = KFold(n_splits=len(x_cluster2), shuffle=True, random_state=49)
    predicted_values = []
    sse_values = []
    coef_values = []

    for train_index, test_index in kf2.split(x_cluster2):
        x_train, x_test = x_cluster2[train_index], x_cluster2[test_index]
        y_train, y_test = y_cluster2[train_index], y_cluster2[test_index]

        lregr = LinearRegression()
        lregr.fit(x_train, y_train)

        y_pred = lregr.predict(x_test)
        sse_values.append(np.mean((y_test - y_pred)**2))

        coef_values.append(lregr.coef_)
        predicted_values.append(y_pred.flatten())
        table = np.column_stack((y_test, y_pred))

    sse = np.mean(sse_values)
    ss_tot = sum((y_cluster2 - np.mean(y_cluster2))**2)/len(y_cluster2)
    r_squared = 1 - sse/ss_tot
    r_squared_values_m2.append(r_squared)

    mean_coef = np.mean(coef_values, axis=0)

#join in a table with columns: threshold, r_squared_m1, r_squared_m2
df = pd.DataFrame({'threshold':thresholds, 'r_squared_m1':r_squared_values_m1, 'r_squared_m2':r_squared_values_m2})
print("FINAL TABLE")
#best threshold is the one with the highest sum of r_squared_m1 and r_squared_m2
df['sum'] = df['r_squared_m1'] + df['r_squared_m2']
df = df.sort_values(by=['sum'], ascending=True)

print(df)



In [ ]:
ransac = RANSACRegressor(estimator = LinearRegression(), min_samples=5, max_trials=500_000, random_state=42)
#nota: residual_threshold= x is the maximum distance for a sample to be classified as an inlier
#nota: residual_threshold = 1 is the default 
# mano nao sejas mau, dá la undeafen        #ok, queres que calcule isso? tanto q o default q eles usam é o MAD(median absolute deviation)
ransac.fit(x, y)
#residual factor used? 
print("Residual factor:",ransac.residual_threshold)

inlier_mask = ransac.inlier_mask_
inliers = []
for i in range(len(inlier_mask)):
    if inlier_mask[i] == True:
        inliers.append(i)
print("Inliers:",inliers)
print("Inliers len:",len(inliers))

outliers  = []
for i in range(len(inlier_mask)):
    if inlier_mask[i] == False:
        outliers.append(i)
print("Outliers:",outliers)
print("Outliers len:",len(outliers))

x_inliers = x[inliers]
y_inliers = y[inliers]
#print(ransac.score(x_inliers, y_inliers)

# Get the outlier mask (points that do not fit the best model)
outlier_mask = np.logical_not(inlier_mask)

# Extract the coefficients of the best model (intercept and slopes for each feature)
intercept = ransac.estimator_.intercept_
coefficients = ransac.estimator_.coef_

print("Coefs:",coefficients.flatten())
print("Score for inliers:")
print(ransac.score(x_inliers, y_inliers))
print("Score for outliers:")
print(ransac.score(x[outliers], y[outliers]))

model1 = ransac
model1_coef = coefficients.flatten()
model1_indexes = x_inliers[:,1]


In [ ]:
x_outliers = x[outliers]
y_outliers = y[outliers]

# make regression line for outliers
lin = LinearRegression()
lin.fit(x_outliers, y_outliers)
y_pred = lin.predict(x_outliers)
print("Score for outliers with linear regression:")
print(lin.score(x_outliers, y_outliers))


#now, trying leave on out cross validation
loo = LeaveOneOut()
predicted_values = []
sse_values = []
coef_values = []

for train_index, test_index in loo.split(x_outliers):
    x_train, x_test = x_outliers[train_index], x_outliers[test_index]
    y_train, y_test = y_outliers[train_index], y_outliers[test_index]

    lregr = LinearRegression()
    lregr.fit(x_train, y_train)

    y_pred = lregr.predict(x_test)
    sse_values.append(np.mean((y_test - y_pred)**2))

    coef_values.append(lregr.coef_)
    predicted_values.append(y_pred.flatten())

#print list sses for all iterations
print("SSE values for all iterations:",sse_values)
print("max sse:",max(sse_values))

mean_coef = np.mean(coef_values, axis=0)
mse = np.mean((y_outliers-predicted_values)**2)
var_y = sum((y_outliers - np.mean(y_outliers))**2)/len(y_outliers)
r_squared = 1 - mse/var_y

print(mean_coef[0], f"\nmse is: {mse}", f"\nr^2 is: {r_squared.flatten()}")

model2 = lin
